# AI Agent 개발하기 (1) 

## AI Agent의 구성요소 

AI Agent란 특정 목표를 달성하기 위해 환경과 상호작용하며 자율적으로 행동하는 인공지능 시스템입니다.

LLM(Large Language Model, 예: ChatGPT)은  주어진 텍스트 입력에 대해 자연어 처리를 통해 응답을 생성하는 모델로, 주로 대화나 텍스트 생성에 특화되어 있으며, 환경과의 지속적인 상호작용보다는 주어진 입력에 대한 응답 생성에 중점을 둡니다. 하지만 AI Agent는 환경과 지속적으로 상호작용하며, 지각, 두뇌, 행동 요소를 가집니다.

AI Agent는 3가지 요소로 구성되어 있습니다.

<img src="../resources/assets/agents-components.png" width="40%">

1. Perception (지각): AI 에이전트가 외부 환경으로부터 정보를 수집하고 인식하는 과정.
2. Brain (두뇌): 수집된 정보를 처리하고 의사결정을 내리는 AI 에이전트의 중앙 처리 시스템.
3. Action (행동): 두뇌에서 내린 의사결정에 따라 AI 에이전트가 환경에 반응하고 행동을 수행하는 과정.



## LLM Model API 가져오기

```text
# cd ../
OPENAI_API_KEY=sk-prod-xxx
````

In [80]:
from openai import OpenAI, AsyncOpenAI
import dotenv
import os
import platform
from typing import List, Dict


if not dotenv.load_dotenv("../.env"):
    raise ValueError("failed to load .env file. check your .env file")
    
client = AsyncOpenAI(
    api_key=os.environ.get('OPENAI_API_KEY')
)

## 1. 프롬프트 작성하기

[Generating Code](https://www.promptingguide.ai/applications/coding)을 기반으로 작성하였습니다 

````markdown
assistant>>>
You are a helpful code assistant. 
- Do not explain the code. 
- Only generate the code block.
- Include code to display the output (stdout) for result verification.
- Programming language: {language}. 
- Current platform: {platform.platform()}

[Output format]
```{shell}

```

user>>>

````

In [186]:
def prompt_for_generating_code(user_message:str, language:str):
    return [
        {
            "role": "assistant",
            "content": 
f"""You are a helpful code assistant. 
- Do not explain the code. 
- Only generate the code block.
- Include code to display the output (stdout) for result verification.
- Programming language: {language}. 
- Current platform: {platform.platform()}

[Output format]
```{language}

```
"""
        },
        {
            "role": "user",
            "content": user_message            
        }
    ]

def extract_code_block(text: str, language:str) -> str:
    import re
    if match := re.search(fr'```{language}\s+([\s\S]*?)\s+```', text):
        return match.group(1).strip()
    
    raise ValueError("Fail to find code block")

### 테스트 질문

1. jupyter notebook을 실행하고 있는 pid를 찾아주세요
2. jupyter notebook의 ip와 port를 가져와 주세요

In [193]:
TEST_PROMPT1 = 'jupyter notebook을 실행하고 있는 pid를 찾아주세요'
TEST_PROMPT2 = '현재 동작중인 jupyter notebook의 ip와 port를 가져와 주세요'

In [194]:
async def call_openai_chat(
    messages:List[Dict], 
    model="gpt-4o"
):
    output = await client.chat.completions.create(
        model=model, messages=messages
    )    
    return output.choices[0].message.content

In [195]:
programming = 'shell'

messages = prompt_for_generating_code(
    TEST_PROMPT2, 
    programming
)

output = await call_openai_chat(messages=messages)

extract_code_block(output, programming)

'jupyter notebook list'

In [198]:
extract_code_block(output, programming)

'jupyter notebook list'

### 실제 Shell Script을 실행하는 코드

In [134]:
def run_shell_script(shell_script: str):
    import subprocess
    result = subprocess.run(
        shell_script, 
        shell=True, 
        capture_output=True, 
        text=True
    )
    return {
        "returncode": result.returncode,
        "stdout": result.stdout,
        "stderr": result.stderr
    }

## ChatGPT가 반환한 결과 프롬프트들

ChatGPT의 프롬프트가 종종 실패할 수 있습니다. 이 경우 대체 프롬프트를 생성해서 실행해주어야 합니다.

1. 첫번째 시도 > 실패
````shell
pgrep jupyter
````

2. 두번째 시도 > 성공

````shell
ps -ax | grep jupyter-notebook | grep -v grep | awk '{print $1}'

>>> 16630
````

3. 세번째 시도 > 성공

````shell
!ps aux | grep jupyter | grep -v grep | awk '{print $2}'

>>> 16630
````

4. 네번째 시도 > 성공

````shell
jupyter_pid=$(pgrep -f "jupyter-notebook")\necho "Jupyter Notebook PID: $jupyter_pid"

>>> Jupyter Notebook PID:
````

#### 첫번째 시도

In [164]:
# 첫 번째 시도
code = 'pgrep jupyter'
print(f"IN>\n{code}\n\nOUT>\n{run_shell_script(code)}")

IN>
pgrep jupyter

OUT>
{'returncode': 1, 'stdout': '', 'stderr': ''}


#### 두번째 시도

In [165]:
code = "ps -ax | grep jupyter-notebook | grep -v grep | awk '{print $1}'"
print(f"IN>\n{code}\n\nOUT>\n{run_shell_script(code)}")

IN>
ps -ax | grep jupyter-notebook | grep -v grep | awk '{print $1}'

OUT>
{'returncode': 0, 'stdout': '16630\n', 'stderr': ''}


#### 세번째 시도

In [166]:
code = "ps aux | grep jupyter | grep -v grep | awk '{print $2}'"
print(f"IN>\n{code}\n\nOUT>\n{run_shell_script(code)}")

IN>
ps aux | grep jupyter | grep -v grep | awk '{print $2}'

OUT>
{'returncode': 0, 'stdout': '16630\n', 'stderr': ''}


#### 네번째 시도

In [167]:
code = '''jupyter_pid=$(pgrep -f "jupyter-notebook")\necho "Jupyter Notebook PID: $jupyter_pid"'''
print(f"IN>\n{code}\n\nOUT>\n{run_shell_script(code)}")

IN>
jupyter_pid=$(pgrep -f "jupyter-notebook")
echo "Jupyter Notebook PID: $jupyter_pid"

OUT>
{'returncode': 0, 'stdout': 'Jupyter Notebook PID: \n', 'stderr': ''}


### 실패한 케이스에 대해 재반복하는 프롬프트 작성


In [ ]:
def prompt_for_generating_code(user_message:str, language:str):
    return [
        {
            "role": "assistant",
            "content": 
f"""You are a helpful code assistant. 
- Do not explain the code. 
- Only generate the code block.
- Include code to display the output (stdout) for result verification.
- Programming language: {language}. 
- Current platform: {platform.platform()}

[Output format]
```{language}

```
"""
        },
        {
            "role": "user",
            "content": user_message            
        }
    ]

def extract_code_block(text: str, language:str) -> str:
    import re
    if match := re.search(fr'```{language}\s+([\s\S]*?)\s+```', text):
        return match.group(1).strip()
    
    raise ValueError("Fail to find code block")